In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from math import sqrt 
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
from catboost import CatBoostRegressor
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import time

In [3]:
ROOT = "/root/air/"
train = pd.read_csv(ROOT+ "data/train-proc-v2.csv")
test = pd.read_csv(ROOT+ "data/test-proc-v2.csv")

In [4]:
test.drop("target",axis  = 1,inplace  = True)

In [5]:
import logging
import random
import os
seed = 1337

In [6]:
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

In [7]:
log_filename = "gridf" + '.out'
logging.basicConfig(filename="/root/air/log/"+log_filename, level=20)
logging.info('gridf {}')

In [8]:
from sklearn.preprocessing import LabelEncoder
lab  = LabelEncoder()

In [9]:
lab.fit(train["location"])

LabelEncoder()

In [10]:
train.location  = lab.transform(train.location)
test.location  = lab.transform(test.location)

In [11]:
X=train.drop(["ID",'target'],axis =1)
y=train.target

In [12]:
from helper import vars_cat, catboost_helper, write_csv, catboost_helper_fold

In [13]:
for i in range(25, 50, 3):
    logging.info('fold {}'.format(i))
    start_time = time.time()
    cat_vars = ['min_atmos_press', 'median_atmos_press', 'mean_atmos_press', 'median_wind_spd', 'newtemp64', 'atmospherepressure33', 'min_temp', 'atmospherepressure56', 'median_wind_dir', 'atmospherepressure92', 'max_wind_spd', 'min_wind_spd', 'mean_wind_dir', 'max_rel_humidity', 'newtemp16', 'mean_precip', 'atmospherepressure105', 'max_wind_dir', 'ptp_wind_spd', 'var_wind_dir', 'median_temp', 'newtemp39', 'ptp_atmos_press', 'newtemp0', 'max_temp', 'newtemp23', 'windspeed119', 'location', 'newtemp92', 'atmospherepressure34', 'newtemp57', 'newtemp2', 'atmospherepressure104', 'min_wind_dir', 'atmospherepressure32', 'atmospherepressure20', 'std_wind_dir', 'windspeed95', 'newtemp103', 'ptp_precip', 'atmospherepressure8', 'var_rel_humidity', 'newtemp73', 'newtemp104', 'newtemp49', 'newtemp25', 'atmospherepressure86', 'min_rel_humidity', 'ptp_rel_humidity', 'newrel_humidity34', 'atmospherepressure2', 'newtemp29', 'ptp_wind_dir', 'windspeed47', 'newrel_humidity32', 'windspeed71', 'newtemp3', 'newwind_dir103', 'atmospherepressure12', 'max_precip', 'var_temp', 'max_atmos_press', 'atmospherepressure44', 'std_atmos_press', 'newtemp81', 'newtemp115', 'std_precip', 'std_rel_humidity', 'windspeed120', 'atmospherepressure116', 'newtemp120', 'newrel_humidity2', 'newrel_humidity0', 'newrel_humidity114', 'newtemp58', 'newtemp66', 'newwind_dir56', 'atmospherepressure87', 'newrel_humidity11', 'ptp_temp', 'newrel_humidity78', 'atmospherepressure98', 'newrel_humidity8', 'median_rel_humidity', 'mean_temp', 'newtemp74', 'atmospherepressure9', 'newtemp54', 'newtemp47', 'newtemp52', 'newrel_humidity59', 'newtemp50', 'atmospherepressure7', 'std_temp', 'newwind_dir90', 'newwind_dir54', 'atmospherepressure3', 'mean_wind_spd', 'newrel_humidity10', 'mean_rel_humidity', 'newtemp101', 'newwind_dir3', 'newrel_humidity1', 'newtemp41', 'newwind_dir120', 'newrel_humidity95', 'atmospherepressure85', 'atmospherepressure62', 'newtemp102', 'newrel_humidity6', 'atmospherepressure68', 'newwind_dir29', 'newrel_humidity120', 'newtemp20', 'newtemp87', 'var_precip', 'atmospherepressure103', 'newtemp119', 'newwind_dir102', 'var_atmos_press', 'newtemp78', 'newtemp89', 'newrel_humidity41', 'newrel_humidity48', 'windspeed115', 'newtemp95', 'newtemp112', 'newwind_dir53', 'atmospherepressure50', 'windspeed108', 'newrel_humidity50', 'newrel_humidity102', 'std_wind_spd', 'atmospherepressure39', 'newtemp30', 'newwind_dir78', 'newtemp90', 'windspeed107', 'atmospherepressure63', 'newtemp32', 'newtemp75', 'windspeed31', 'newtemp48', 'windspeed4', 'newtemp1', 'windspeed80', 'newwind_dir52', 'atmospherepressure109', 'newtemp97', 'newtemp85', 'windspeed49', 'newwind_dir32', 'newtemp63', 'windspeed56', 'windspeed5', 'newtemp76', 'newrel_humidity103', 'atmospherepressure1', 'newtemp70', 'newrel_humidity17', 'newtemp96', 'atmospherepressure25', 'windspeed42', 'newrel_humidity26', 'atmospherepressure120', 'newrel_humidity61', 'newtemp77', 'newtemp46', 'windspeed97', 'newrel_humidity98', 'newtemp72', 'newrel_humidity33', 'newrel_humidity115', 'newtemp15', 'windspeed32', 'atmospherepressure10']
    result_cat, err = catboost_helper_fold(cat_vars, i, X, y, test)
    logging.info('err {0:.2f}'.format(err))
    logging.info('time taken {}'.format(start_time - time.time()))
    write_csv(test, result_cat, err)

0:	learn: 41.1768209	test: 41.1768209	test1: 42.6795365	best: 42.6795365 (0)	total: 66.6ms	remaining: 7m 45s
100:	learn: 26.1923556	test: 26.1923556	test1: 28.6264710	best: 28.6264710 (100)	total: 1.93s	remaining: 2m 11s
200:	learn: 22.5481888	test: 22.5481888	test1: 26.5548069	best: 26.5548069 (200)	total: 4.19s	remaining: 2m 21s
300:	learn: 20.2189458	test: 20.2189458	test1: 25.5673246	best: 25.5673246 (300)	total: 6.27s	remaining: 2m 19s
400:	learn: 18.4670035	test: 18.4670035	test1: 24.9488714	best: 24.9426996 (399)	total: 8.95s	remaining: 2m 27s
500:	learn: 17.0122104	test: 17.0122104	test1: 24.5744704	best: 24.5579807 (487)	total: 11.6s	remaining: 2m 30s
600:	learn: 15.7082848	test: 15.7082848	test1: 24.2888880	best: 24.2796527 (595)	total: 14.4s	remaining: 2m 33s
700:	learn: 14.5976001	test: 14.5976001	test1: 23.9983591	best: 23.9959677 (692)	total: 17s	remaining: 2m 32s
800:	learn: 13.6676079	test: 13.6676079	test1: 23.8296440	best: 23.8262847 (786)	total: 19.5s	remaining: 2m 3